In [1]:
import pyspark

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    
type(sc) 

pyspark.context.SparkContext

In [14]:
def tupla(valor):
    
    if (x > 5):
        
        return (tuple(x,1))
    
    else:
        
        return (tuple(x,0))
##########################################################################
lista = [3,5,8,2,5,6,8,0,1,34]

rdd = sc.parallelize(lista,3)

rdd.map(lambda x: (x,1) if x > 5 else (x,0))\
   .filter(lambda x: x[1] == 1).collect()

[(8, 1), (6, 1), (8, 1), (34, 1)]

In [3]:
from operator import add
#Resolucion del examen recuperatortio
#Ejercicio 1

censo = [('bs.as', 'cf', 20000), 
         ('santa fe', 'rosario', 30000),
         ('bs.as', 'mar del plata', 10000),
         ('bs.as', 'la matanza', 40000),
         ('jujuy', 'tilcara', 3000),
         ('santa fe', 'santa fe', 25000),
         ('santa fe', 'colon', 31000)]

rdd = sc.parallelize(censo)
print((rdd.filter(lambda x: (20000 <= x[2] <= 50000))\
   .map(lambda x: (x[0], 1))\
   .reduceByKey(add)\
   .reduce(lambda x, y: x if (x[1] > y[1]) else y)))


('santa fe', 3)


In [4]:
rdd.takeSample(True,2)

[('bs.as', 'cf', 20000), ('santa fe', 'rosario', 30000)]

rdd.takeSample(True,3)

In [5]:
#Ejercicio 2 del recuperatorio
# 1 parte
grados = [('alfa', 2011, 12, 12, 12, 33, 20), 
          ('beta', 2011, 2, 30, 14, 52, 18),
          ('alfa', 2014, 12, 12, 15, 12, 23),
          ('alfa', 2014, 12, 12, 5, 12, 10), 
          ('beta', 2001, 2, 31, 14, 52, -3),
          ('beta', 2001, 2, 29, 14, 52, -10),
          ('alfa', 2013, 13, 13, 23, 7, -40)]

#resultado (('alfa', 123), 21.5)
from operator import add

rdd = sc.parallelize(grados)
rdd2 = sc.parallelize([])

rdd = rdd.map(lambda x: ((x[0], x[1], x[2], x[3]), (x[6], 1)))\
         .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
         .map(lambda x: (x[0], x[1][0]/x[1][1]))
        
rdd2 = rdd

rdd = rdd.map(lambda x:((x[0][3] + 1, x[0][0]), x[1]))
rdd2 = rdd2.map(lambda x: ((x[0][3], x[0][0]), x[1]))
print(rdd.join(rdd2)\
   .map(lambda x: (x[0], abs(x[1][1] - x[1][0])))\
   .reduce(lambda x, y: x if (x[1] > y[1]) else y))
   

#   .reduceByKey(lambda x, y: (abs(y[1] - x[1]))).collect()


((13, 'alfa'), 60)


In [6]:
from operator import add
rdd = sc.parallelize([("a", (1,3)), ("b", (0,2)), ("a", (1,1))])
rdd.reduceByKey(add).collect()

[('a', (1, 3, 1, 1)), ('b', (0, 2))]

In [7]:
#Tenemos información sobre las compras que los usuarios realizan en un
#sitio online, cada registro indica el usuario, el id de la operación y 
#una lista de productos comprados, esta información se encuentra en un RDD
#llamado "purchases" (operation_id, (user_id, [product_ids])).
#Queremos construir una lista de productos comprados frecuentemente junto
#con otros, para eso queremos construir un nuevo RDD de la forma 
#(product_id,[p_id1,p_id2,p_id3]) donde p_id1,p_id2 y p_id3 son los tres 
#productos que mas frecuentemente se compran junto con el product_id.


def func2(lista1, lista2):
    
    lista_r = []
    
    for e in lista1:
        
        if e in lista2:
            
            lista_r.append(e)
            
            for i in lista2:
                
                if i != e:
                    
                    lista_r.append(i)
    return lista_r
    


#def funcion(lista2,e)
#lista_r = []
#  if e in lista2:
#    if i != e:
#      lista_r.append(i)
# return (e,lista_r)

#lista_r = []
#for e in lista1:
#  if e in lista2:
#   lista_r.append(e)
#   for i in lista1
#    if i != e:
#      lista_r.append(i)
# return lista_r
#########################################################################

compras = [(23, (2, [1,2,3])),
           (3, (22, [3,5,1])),
           (33, (1, [34])),
           (9, (7, [4,5,6])),
           (13, (8, [1,3,5])),
           (123, (9, [6,7,8])),
           (100, (30, [1,2,3,4,5,6,7,8,9]))]

rdd = sc.parallelize(compras)
rdd.map(lambda x: (x[1][1]))\
   .map(lambda x, y: func2(x,y)).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 9.0 failed 1 times, most recent failure: Lost task 2.0 in stage 9.0 (TID 46, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/alex92/Documentos/Datos/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/alex92/Documentos/Datos/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/alex92/Documentos/Datos/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
TypeError: <lambda>() takes exactly 2 arguments (1 given)

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonR

In [ ]:
#(id_batalla, (nombre_medabot, [lista_de_medabots_vencidos]))
#quiero una lista de mayor a menor de los nombres de los 5 medabots 
#que mas veces fueron derrotados

medabatallas = [(23, ('Robo-Emperador', [123,212,423,456])),
                (22, ('brass', [456, 423])), 
                (212, ('metabee', [283, 973, 989, 456])),
                (100, ('Mega-Emperador', [283, 423])),
                (34, ('art-bettle', [989, 456]))]
rdd2 = sc.parallelize(medabatallas)

medabots = [('rokusho', 123, 'kbt', 90),
            ('art-bettle', 789, 'kbt',95),
            ('sumilidon', 283, 'smd', 88),
            ('brass', 212, 'brs', 40),
            ('worbandi', 788, 'war', 98),
            ('metabee', 423,'kbt', 90),
            ('Robo-Emperador', 973, 'gnt', 101),
            ('Mega-Emperador', 989, 'gnt', 110),
            ('totalaiser', 456, 'tot', 63)]

rdd = sc.parallelize(medabots)

from operator import add

rdd = rdd.map(lambda x: (x[1], x[0]))

rdd2.flatMap(lambda x:x[1][1])\
#    .map(lambda x: (x,1))\
#    .reduceByKey(add)\
#    .join(rdd)\
#    .map(lambda x: (x[1][1], x[1][0]))\
#    .takeOrdered(3, key = lambda x: -x[1])#esta linea reemplaza a las 2 de abajo
#    .sortBy(lambda x: -x[1])\
#    .take(3)
    


#variante: añadir el factor empate ej: metabee y worbandi
#añadir una lista para cada medabots, junto con los 3 primeros que mas veces
#aparece en una lista,como el ej de arriba
#otra jugar con el modelo kbt etc

In [ ]:
medabots = [('rokusho', 123, 'kbt', 90),
            ('art-bettle', 789, 'kbt',95),
            ('sumilidon', 283, 'smd', 88),
            ('brass', 212, 'brs', 40),
            ('worbandi', 788, 'war', 98),
            ('metabee', 423,'kbt', 90),
            ('Robo-Emperador', 973, 'gnt', 101),
            ('Mega-Emperador', 989, 'gnt', 110),
            ('totalaiser', 456, 'tot', 63)]

rdd = sc.parallelize(medabots)


   .     

In [ ]:
#Un telescopio registra automaticamente la luminosidad de distintas
#estrellas generando un RDD con registros de tipo
#(star_id, magnitude,spectral_type, timestamp). Queremos obtener un listado
#de estrellas que tienen el mismo tipo espectral e igual promedio de 
#magnitud una vez redondeado el mismo a un decimal. El resultado debe ser 
#una lista en donde cada elemento de la lista es una lista de ids de 
#estrellas similares.

stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

rdd = sc.parallelize(stars)

rdd.map(lambda x: (x[0], (x[2], x[1], 1)))\
   .reduceByKey(lambda x, y: (x[0], x[1] + y[1], x[2] + y[2]))\
   .map(lambda x: ((x[1][0], x[1][1] / x[1][2]), x[0]))\
   .groupByKey().mapValues(lambda x: list(x))\
   .map(lambda x: (list(x[1]))).collect()

In [ ]:
#En este ejercicio queremos programar un sistema que recomiende
#textos a usuarios en base a sus gustos sobre ciertos términos (palabras).
#Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
#es un string de longitud variable. Además contamos con un RDD que
#indica qué términos le gustan o no a cada usuario de la forma (userId,
#término, score) por ejemplo (23, “calesita”, -2). Se pide programar en
#Spark un programa que calcule el score total de cada documento para
#cada usuario generando un RDD de la forma (userId, docId, score) en
#donde el score es simplemente la suma de los scores del usuario para
#los términos que aparecen en el documento. Puede haber términos en
#los documentos para los cuales no exista score de algunos usuarios, en
#estos casos simplemente los consideramos neutros (score=0).  

def func(unaTupla):
    
    print(unaTupla)
    lista = []
    
    for t in unaTupla[0]:
        
        lista.append((t, unaTupla[1]))
        
     
    return lista 
    

####################################################

textos = [(3, 'futbol en los mundiales'),
          (6, 'cocinar y limpiar'),
          (9, 'economia y politica'),
          (65, 'java el lenguaje del futuro')]

terminos =[(23, 'cocinar', -4),
           (24, 'futbol', 10),
           (17, 'limpiar', 8),
           (24, 'limpiar', -1),
           (17, 'futbol', 1),
           (24, 'mundiales', 9),
           (24, 'economia', 2),
           (24, 'politica', 3),
           (23, 'java', 10)]

# Gaston: def funcion(tu)
#lista = []
#for t in tu[0]:
#  lista.append((t,tu[1]))
#return lista                        
#Gaston: donde tu es la tupla tuya
from operator import add

rdd1 = sc.parallelize(textos)
rdd2 = sc.parallelize(terminos)

rdd1 = rdd1.map(lambda x: ((x[1].split(' ')), x[0]))\
           .map(lambda x: func(x))\
           .flatMap(lambda x: x)
        
rdd2.map(lambda x: (x[1], (x[0], x[2])))\
    .join(rdd1)\
    .map(lambda x: ((x[1][0][0], x[1][1]), x[1][0][1]))\
    .reduceByKey(add).collect()

#[(('futbol', 'en', 'los', 'mundiales'), 3)]
#[(('futbol', 3), ('en', 3), ('los', 3), ('mundiales', 3))]

In [ ]:
x = sc.parallelize([("a", 1), ("b", 4), ('c',3)])
y = sc.parallelize([("a", ('12', 4)), ("a", ('papa', '333')), ('c',5), ('a', 10)])
print(x.join(y).collect())

In [15]:
censo = [('bs.as', 'cf', 20000), 
         ('santa fe', 'rosario', 30000),
         ('bs.as', 'mar del plata', 10000),
         ('bs.as', 'la matanza', 40000),
         ('jujuy', 'tilcara', 3000),
         ('santa fe', 'santa fe', 25000),
         ('santa fe', 'colon', 31000)]

rdd = sc.parallelize(censo)

rdd.map(lambda x: (x[0], x[2]))\
   .reduceByKey(lambda x, y: x if (x > y) else y)

PythonRDD[48] at RDD at PythonRDD.scala:43